In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

from UHF import *

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})


## Parameters

In [2]:
basis = "STO-3G"
N_H = 3
N_a = 2
N_b = 1

In [3]:
mol = gqcpy.Molecule.HChain(N_H, 1.4, 0)

In [4]:
_, HFpar = UHF(mol, N_a, N_b, basis)

UHF energy:  -1.5095578776455065


In [5]:
spinor_basis = gqcpy.USpinOrbitalBasis_d(mol, basis)

## Testing

In [6]:
S_aa = spinor_basis.quantize(gqcpy.OverlapOperator()).alpha.parameters()
S_bb = spinor_basis.quantize(gqcpy.OverlapOperator()).beta.parameters()

In [7]:
Sp_a =  np.zeros_like(S_aa)
Sp_b = S_aa

In [8]:
Sm_a = S_bb
Sm_b = np.zeros_like(S_aa)

In [9]:
Spm_a = np.zeros_like(S_aa)
Spm_b = S_aa

In [10]:
Sz2_a = 0.25 * S_aa
Sz2_b = 0.25 * S_bb

In [11]:
Sz_a = spinor_basis.quantize(gqcpy.ElectronicSpin_zOperator()).alpha.parameters()
Sz_b = spinor_basis.quantize(gqcpy.ElectronicSpin_zOperator()).beta.parameters()

In [12]:
spin_a = Sz_a + Spm_a + Sz2_a
spin_b = Sz_b + Spm_b + Sz2_b

spin = gqcpy.ScalarUSQOneElectronOperator_d(gqcpy.ScalarUSQOneElectronOperatorComponent_d(spin_a), gqcpy.ScalarUSQOneElectronOperatorComponent_d(spin_b))
Sz = gqcpy.ScalarUSQOneElectronOperator_d(gqcpy.ScalarUSQOneElectronOperatorComponent_d(Sz_a), gqcpy.ScalarUSQOneElectronOperatorComponent_d(Sz_b))


In [13]:
correct = np.einsum('pr,qs->pqrs', S_aa, S_bb)

In [14]:
# tensor_aa = gqcpy.ScalarPureUSQTwoElectronOperatorComponent_d(np.einsum('pq,rs->pqrs', Sz_a, Sz_b))
# tensor_bb = gqcpy.ScalarPureUSQTwoElectronOperatorComponent_d(np.einsum('pq,rs->pqrs', Sz_b, Sz_a))

tensor_aa = gqcpy.ScalarPureUSQTwoElectronOperatorComponent_d(np.zeros((N_H, N_H, N_H, N_H)))
tensor_bb = gqcpy.ScalarPureUSQTwoElectronOperatorComponent_d(np.zeros((N_H, N_H, N_H, N_H)))

tensor_ab = gqcpy.ScalarMixedUSQTwoElectronOperatorComponent_d(-1 * correct)
tensor_ba = gqcpy.ScalarMixedUSQTwoElectronOperatorComponent_d(-1 * np.einsum('pr,qs->pqrs', S_bb, S_aa))
tensor = gqcpy.ScalarUSQTwoElectronOperator_d(tensor_aa, tensor_ab, tensor_ba, tensor_bb)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. gqcpy.ScalarPureUSQTwoElectronOperatorComponent_d(array: Eigen::Tensor<double, 4, 0, long>)

Invoked with: array([[[[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000]],

        [[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000]],

        [[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000]]],


       [[[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000]],

        [[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000]],

        [[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000]]],


       [[[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000]],

        [[0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000],
         [0.000, 0.000, 0.000

In [ ]:
tensor.alphaBeta().parameters()

In [ ]:
correct = np.einsum('pr,qs->pqrs', S_aa, S_bb)

In [ ]:
np.einsum('pq,rs->pqrs', S_aa, S_bb).transpose(0, 2, 1, 3)

In [ ]:
OneDM = HFpar.calculateScalarBasis1DM()
twoDM = HFpar.calculateScalarBasis2DM()

In [ ]:
spin.calculateExpectationValue(OneDM)[0]

In [ ]:
tensor.calculateExpectationValue(twoDM)[0]

In [ ]:
DM_test_2 = HFpar.calculateScalarBasis2DM().alphaBeta().tensor()

In [ ]:
np.einsum("pqrs, pqrs-> ", np.einsum('pr,qs->pqrs', S_aa, S_bb), DM_test_2)

In [ ]:
S2_val = spin.calculateExpectationValue(OneDM)[0] + tensor.calculateExpectationValue(twoDM)[0] 
S2_val

In [ ]:
Sz_val = Sz.calculateExpectationValue(OneDM)[0]
Sz_val

In [ ]:
N_b - np.trace(OneDM.alpha.matrix() @ S_aa @ OneDM.beta.matrix() @ S_bb)

In [ ]:
# np.trace(OneDM.alpha.matrix() @ S_aa @ OneDM.beta.matrix() @ S_bb)

In [ ]:
occ_a = N_a
occ_b = N_b

occ_indx_a = np.arange(occ_a)  # indices of the occupied alpha orbitals
occ_indx_b = np.arange(occ_b)  # indices of the occupied beta orbitals

occ_a_orb = HFpar.expansion().alpha.matrix()[:, occ_indx_a]  # orbital coefficients associated with occupied alpha orbitals
occ_b_orb = HFpar.expansion().beta.matrix()[:, occ_indx_b]  # orbital coefficients associated with occupied beta orbitals

s = occ_a_orb.conj().T @ S_aa @ occ_b_orb  # Basically (alpha orbitals).T * S * (beta orbitals)

ss_xy = (occ_a + occ_b) * 0.5 #- np.einsum('ij,ij->', s.conj(), s)  # = S^2_x + S^2_y
ss_z = (occ_a - occ_b)**2 * 0.25  # = S^2_z
ss = (ss_xy + ss_z).real  # = S^2_total
s_z = (occ_a - occ_b) / 2  # = S_z

In [ ]:
np.einsum('ij,ij->', s.conj(), s)

In [ ]:
ss_xy

In [ ]:
ss_z

In [ ]:
ss

In [ ]:
HFpar.calculateOrthonormalBasis2DM().betaAlpha().tensor()

In [ ]:
# aa = gqcpy.PureSpinResolved2DMComponent_d(np.zeros((N_H, N_H, N_H, N_H)))
# bb = gqcpy.PureSpinResolved2DMComponent_d(np.zeros((N_H, N_H, N_H, N_H)))

# ab = gqcpy.MixedSpinResolved2DMComponent_d(-1 * correct)
# ba = gqcpy.MixedSpinResolved2DMComponent_d(-1 * np.einsum('pr,qs->pqrs', S_bb, S_aa))
